<a href="https://colab.research.google.com/github/farhanfuadabir/SHL2020/blob/master/SHL_featuresExtraction_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import pickle as pk
from scipy import signal,stats
from scipy.special import entr
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

##Data **Import**

In [ ]:
location = '/content/drive/My Drive/SHL Dataset 2020/challenge-2020-train-hand/'

acc_x = pd.read_csv(location + 'Acc_x.txt', header = None, sep=" ").to_numpy()
print("Acc_x Import Done")
acc_y = pd.read_csv(location + 'Acc_y.txt', header = None, sep=" ").to_numpy()
print("Acc_y Import Done")
acc_z = pd.read_csv(location + 'Acc_z.txt', header = None, sep=" ").to_numpy()
print("Acc_z Import Done")
mag_x = pd.read_csv(location + 'Mag_x.txt', header = None, sep=" ").to_numpy()
print("Mag_x Import Done")
mag_y = pd.read_csv(location + 'Mag_y.txt', header = None, sep=" ").to_numpy()
print("Mag_y Import Done")
mag_z = pd.read_csv(location + 'Mag_z.txt', header = None, sep=" ").to_numpy()
print("Mag_z Import Done")
gyr_x = pd.read_csv(location + 'Gyr_x.txt', header = None, sep=" ").to_numpy()
print("Gyr_x Import Done")
gyr_y = pd.read_csv(location + 'Gyr_y.txt', header = None, sep=" ").to_numpy()
print("Gyr_y Import Done")
gyr_z = pd.read_csv(location + 'Gyr_z.txt', header = None, sep=" ").to_numpy()
print("Gyr_z Import Done")
pres = pd.read_csv(location + 'Pressure.txt', header = None, sep=" ").to_numpy()
print("Pressure Import Done")

y = pd.read_csv(location + 'Label.txt', header = None, sep=" ").mode(axis=1).to_numpy().flatten()
print("Label Import Done")

print("\n\nImport Completed")

Acc_x Import Done
Acc_y Import Done
Acc_z Import Done
Mag_x Import Done
Mag_y Import Done
Mag_z Import Done
Gyr_x Import Done
Gyr_y Import Done
Gyr_z Import Done
Pressure Import Done
Label Import Done


Import Completed


## Feature **Extraction**

In [ ]:
def magnitude(x,y,z):
  return np.sqrt(x**2 + y**2 + z**2)

def entrop(pk,axis=0):
     pk = pk / np.sum(pk, axis=axis, keepdims=True)
     vec = entr(pk)
     S = np.sum(vec, axis=axis)
     return S

def autocorr(x,axis=0):
    result = np.correlate(x, x, mode='full')
    return result[result.size // 2:]


# Magnitude Calculation
acc_MAG = magnitude(acc_x,acc_y,acc_z)
mag_MAG = magnitude(mag_x,mag_y,mag_z)
gyr_MAG = magnitude(gyr_x,gyr_y,gyr_z)

# Statistical Feature Calculation
acc_mean = np.mean(acc_MAG,axis=1)
acc_std = np.std(acc_MAG,axis=1)
acc_max = np.max(acc_MAG,axis=1)
acc_min = np.min(acc_MAG,axis=1)
mag_mean = np.mean(mag_MAG,axis=1)
mag_std = np.std(mag_MAG,axis=1)
mag_max = np.max(mag_MAG,axis=1)
mag_min = np.min(mag_MAG,axis=1)
gyr_mean = np.mean(gyr_MAG,axis=1)
gyr_std = np.std(gyr_MAG,axis=1)
gyr_max = np.max(gyr_MAG,axis=1)
gyr_min = np.min(gyr_MAG,axis=1)
pres_mean = np.mean(pres,axis=1)
pres_std = np.std(pres,axis=1)
pres_max = np.max(pres,axis=1)
pres_min = np.min(pres,axis=1)

# Frequency Domain Feature Calculation
fs = 100
acc_FREQ,acc_PSD = signal.welch(acc_MAG,fs,nperseg=500,axis=1)
mag_FREQ,mag_PSD = signal.welch(mag_MAG,fs,nperseg=500,axis=1)

# Max PSD value
acc_PSDmax = np.max(acc_PSD,axis=1)
mag_PSDmax = np.max(mag_PSD,axis=1)

# Frequency Entropy
acc_entropy = entrop(acc_PSD,axis=1)
mag_entropy = entrop(mag_PSD,axis=1)

# Frequency Center
acc_fc = np.sum((acc_FREQ*acc_PSD),axis=1) / np.sum(acc_PSD,axis=1)
mag_fc = np.sum((mag_FREQ*mag_PSD),axis=1) / np.sum(mag_PSD,axis=1)

# Autocorrelation Calculation
acc_acr = np.apply_along_axis(autocorr,1,acc_MAG)
print(acc_acr.shape)

acc_features = np.stack((acc_mean,acc_std,acc_max,acc_min,acc_PSDmax,acc_entropy,acc_fc),axis=1)
mag_features = np.stack((mag_mean,mag_std,mag_max,mag_min,mag_PSDmax,mag_entropy,mag_fc),axis=1)
gyr_features = np.stack((gyr_mean,gyr_std,gyr_max,gyr_min),axis=1)
pres_features = np.stack((pres_mean,pres_std,pres_max,pres_min),axis=1)

X = np.concatenate([acc_features,mag_features,gyr_features,pres_features],axis=1)

print("X shape: ",X.shape)
print("y shape: ",y.shape)

print("Feature Extraction Done")

NameError: ignored

## Training: **Decision Tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

#Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)

print("Decision Tree Training Done")

Decision Tree Training Done


In [ ]:
#Making Prediction
y_pred = dtree.predict(X_test)

#Evaluating Algorithm
print("Confusion Matrix: \n\n",confusion_matrix(y_test,y_pred))
print("\n\nReport: \n\n",classification_report(y_test,y_pred))
print("Accuracy: ",accuracy_score(y_test,y_pred))

Confusion Matrix: 

 [[1101   42    0   14   22   24   18    9]
 [  33  950    4   24    9   11   12    3]
 [   0    4   94    0    0    0    1    0]
 [   4   25    1  447    6    5    3    0]
 [  22    6    0    7  689   41   24    6]
 [  36   14    0   10   38  242   18    4]
 [  30   13    0    5   15   22  722   80]
 [  18    7    0    1    9    5   98  710]]


Report: 

               precision    recall  f1-score   support

           1       0.89      0.90      0.89      1230
           2       0.90      0.91      0.90      1046
           3       0.95      0.95      0.95        99
           4       0.88      0.91      0.89       491
           5       0.87      0.87      0.87       795
           6       0.69      0.67      0.68       362
           7       0.81      0.81      0.81       887
           8       0.87      0.84      0.86       848

    accuracy                           0.86      5758
   macro avg       0.86      0.86      0.86      5758
weighted avg       0.86  

## Training: **SVM (linear)**

In [ ]:
svmLin = SVC(kernel='linear',C=1)
svmLin.fit(X_train, y_train)

print("SVM_lin Training Done")

In [ ]:
#Making Prediction
y_pred = svmLin.predict(X_test)

#Evaluating Algorithm
print("Confusion Matrix: \n\n",confusion_matrix(y_test,y_pred))
print("\n\nReport: \n\n",classification_report(y_test,y_pred))
print("Accuracy: ",accuracy_score(y_test,y_pred))

## Training: **SVM (poly)**

In [ ]:
svmPoly = SVC(kernel='poly', degree=8)
svmPoly.fit(X_train, y_train)

print("SVM_poly Training Done")

In [ ]:
#Making Prediction
y_pred = svmPoly.predict(X_test)

#Evaluating Algorithm
print("Confusion Matrix: \n\n",confusion_matrix(y_test,y_pred))
print("\n\nReport: \n\n",classification_report(y_test,y_pred))
print("Accuracy: ",accuracy_score(y_test,y_pred))

##**2019** Test Set

In [ ]:
location = '/content/drive/My Drive/SHL Dataset 2019/Test/Hand/'

acc_x = pd.read_csv(location + 'Acc_x.txt', header = None, sep=" ").to_numpy()
acc_y = pd.read_csv(location + 'Acc_y.txt', header = None, sep=" ").to_numpy()
acc_z = pd.read_csv(location + 'Acc_z.txt', header = None, sep=" ").to_numpy()
mag_x = pd.read_csv(location + 'Mag_x.txt', header = None, sep=" ").to_numpy()
mag_y = pd.read_csv(location + 'Mag_y.txt', header = None, sep=" ").to_numpy()
mag_z = pd.read_csv(location + 'Mag_z.txt', header = None, sep=" ").to_numpy()
gyr_x = pd.read_csv(location + 'Gyr_x.txt', header = None, sep=" ").to_numpy()
gyr_y = pd.read_csv(location + 'Gyr_y.txt', header = None, sep=" ").to_numpy()
gyr_z = pd.read_csv(location + 'Gyr_z.txt', header = None, sep=" ").to_numpy()
pres = pd.read_csv(location + 'Pressure.txt', header = None, sep=" ").to_numpy()

y_test = pd.read_csv(location + 'Label_test.txt', header = None, sep=" ").to_numpy()
y_test,_ = stats.mode(y_test,axis=1,nan_policy='omit')
y_test = y_test.flatten()

print("Import Completed")

Import Completed


In [ ]:
def magnitude(x,y,z):
  return np.sqrt(x**2 + y**2 + z**2)

def entrop(pk,axis=0):
     pk = pk / np.sum(pk, axis=axis, keepdims=True)
     vec = entr(pk)
     S = np.sum(vec, axis=axis)
     return S


# Magnitude Calculation
acc_MAG = magnitude(acc_x,acc_y,acc_z)
mag_MAG = magnitude(mag_x,mag_y,mag_z)
gyr_MAG = magnitude(gyr_x,gyr_y,gyr_z)

# Statistical Feature Calculation
acc_mean = np.mean(acc_MAG,axis=1)
acc_std = np.std(acc_MAG,axis=1)
acc_max = np.max(acc_MAG,axis=1)
acc_min = np.min(acc_MAG,axis=1)
mag_mean = np.mean(mag_MAG,axis=1)
mag_std = np.std(mag_MAG,axis=1)
mag_max = np.max(mag_MAG,axis=1)
mag_min = np.min(mag_MAG,axis=1)
gyr_mean = np.mean(gyr_MAG,axis=1)
gyr_std = np.std(gyr_MAG,axis=1)
gyr_max = np.max(gyr_MAG,axis=1)
gyr_min = np.min(gyr_MAG,axis=1)
pres_mean = np.mean(pres,axis=1)
pres_std = np.std(pres,axis=1)
pres_max = np.max(pres,axis=1)
pres_min = np.min(pres,axis=1)

# Frequency Domain Feature Calculation
fs = 100
acc_FREQ,acc_PSD = signal.welch(acc_MAG,fs,nperseg=500,axis=1)
mag_FREQ,mag_PSD = signal.welch(mag_MAG,fs,nperseg=500,axis=1)

# Max PSD value
acc_PSDmax = np.max(acc_PSD,axis=1)
mag_PSDmax = np.max(mag_PSD,axis=1)

# Frequency Entropy
acc_entropy = entrop(acc_PSD,axis=1)
mag_entropy = entrop(mag_PSD,axis=1)

# Frequency Center
acc_fc = np.sum((acc_FREQ*acc_PSD),axis=1) / np.sum(acc_PSD,axis=1)
mag_fc = np.sum((mag_FREQ*mag_PSD),axis=1) / np.sum(mag_PSD,axis=1)

acc_features = np.stack((acc_mean,acc_std,acc_max,acc_min,acc_PSDmax,acc_entropy,acc_fc),axis=1)
mag_features = np.stack((mag_mean,mag_std,mag_max,mag_min,mag_PSDmax,mag_entropy,mag_fc),axis=1)
gyr_features = np.stack((gyr_mean,gyr_std,gyr_max,gyr_min),axis=1)
pres_features = np.stack((pres_mean,pres_std,pres_max,pres_min),axis=1)

X_test = np.concatenate([acc_features,mag_features,gyr_features,pres_features],axis=1)

print("X shape: ",X.shape)
print("y shape: ",y.shape)

print("Feature Extraction Done")

X shape:  (28789, 22)
y shape:  (28789,)
Feature Extraction Done


In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier()
dtree.fit(X, y)

print("Decision Tree Training Done")

#Making Prediction
y_pred = dtree.predict(X_test)

print(X.shape)
print(y.shape)
print(X_test.shape)
print(y_test.shape)

#Evaluating Algorithm
print("Confusion Matrix: \n\n",confusion_matrix(y_test,y_pred))
print("\n\nReport: \n\n",classification_report(y_test,y_pred))
print("Accuracy: ",accuracy_score(y_test,y_pred))

Decision Tree Training Done
(28789, 22)
(28789,)
(55811, 22)
(55811,)
Confusion Matrix: 

 [[6117  656   12  303 1540 2719  775  852]
 [ 340 4760  426 1166 1400  208  150  355]
 [   4  365 2788  384   20    0    2    1]
 [ 165 4173  269 1091  700  298   39  105]
 [ 586  482    8  333 3820 2110 1007 2989]
 [ 403  240    5   99 1015 1461  508 1810]
 [ 311  135    0   65  483  439 1421 2080]
 [  29   37    0    0    5    5   43 1699]]


Report: 

               precision    recall  f1-score   support

           1       0.77      0.47      0.58     12974
           2       0.44      0.54      0.48      8805
           3       0.79      0.78      0.79      3564
           4       0.32      0.16      0.21      6840
           5       0.43      0.34      0.38     11335
           6       0.20      0.26      0.23      5541
           7       0.36      0.29      0.32      4934
           8       0.17      0.93      0.29      1818

    accuracy                           0.41     55811
   macro 